In [1]:
import os
import re
import json

import numpy as np
import tensorflow as tf

from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# solve the problem of "libdevice not found at ./libdevice.10.bc"
os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/home/r10222035/.conda/envs/tf2'

2024-05-23 10:50:53.531056: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-23 10:50:53.620341: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def get_info(path):
    # path: run path
    name = os.path.split(path)[1]

    with open(os.path.join(path, f'{name}_tag_1_banner.txt')) as f:
        for line in f.readlines():

            #  Integrated weight (pb)  :       0.020257
            match = re.match('#  Integrated weight \(pb\)  : +(\d+\.\d+)', line)
            if match:
                # unit: fb
                cross_section = float(match.group(1)) * 1000
            #  Number of Events        :       100000
            match = re.match('#  Number of Events        :       (\d+)', line)
            if match:
                # unit: fb
                nevent = int(match.group(1))

    return cross_section, nevent

In [3]:
def compute_nevent_in_SR_SB(sensitivity=1.0):
    results_s = np.load('../Sample/HVmodel/data/selection_results_SB_4400_5800_s.npy', allow_pickle=True).item()
    results_b = np.load('../Sample/HVmodel/data/selection_results_SB_4400_5800_b.npy', allow_pickle=True).item()

    # Total cross section and number of events
    xection, _ = get_info('../Sample/ppjj/Events/run_03')

    # cross section in signal region and sideband region
    cross_section_SR = results_b['cutflow_number']['Signal region'] / results_b['cutflow_number']['Total'] * xection
    cross_section_SB = results_b['cutflow_number']['Sideband region'] / results_b['cutflow_number']['Total'] * xection
    print(f'Background cross section, SR: {cross_section_SR:.2f} fb, SB: {cross_section_SB:.2f} fb')

    # number of background events in signal region and sideband region
    L = 139 * 1
    n_SR_B = cross_section_SR * L
    n_SB_B = cross_section_SB * L

    print(f'Background sample size: SR: {n_SR_B:.1f}, SB: {n_SB_B:.1f}')

    n_SR_S = sensitivity * np.sqrt(n_SR_B)
    n_SB_S = n_SR_S * results_s['cutflow_number']['Sideband region'] / results_s['cutflow_number']['Signal region']
    print(f'Signal sample size: SR: {n_SR_S:.1f}, SB: {n_SB_S:.1f}')

    return n_SR_S, n_SR_B, n_SB_S, n_SB_B

In [4]:
def get_SR_SB_sample_from(npy_dirs: list, nevents: tuple, seed=0):
    # npy_dirs: list of npy directories
    # nevents: tuple of (n_sig_SR, n_sig_SB, n_bkg_SR, n_bkg_SB)
    data_SR = None
    label_SR = None

    data_SB = None
    label_SB = None

    data_sig_SR = np.load(os.path.join(npy_dirs[0], 'sig_in_SR-data.npy'))
    data_sig_SB = np.load(os.path.join(npy_dirs[0], 'sig_in_SB-data.npy'))
    data_bkg_SR = np.load(os.path.join(npy_dirs[0], 'bkg_in_SR-data.npy'))
    data_bkg_SB = np.load(os.path.join(npy_dirs[0], 'bkg_in_SB-data.npy'))

    n_sig_SR, n_sig_SB, n_bkg_SR, n_bkg_SB = nevents

    np.random.seed(seed)
    idx_sig_SR = np.random.choice(data_sig_SR.shape[0], n_sig_SR, replace=False)
    idx_sig_SB = np.random.choice(data_sig_SB.shape[0], n_sig_SB, replace=False)
    idx_bkg_SR = np.random.choice(data_bkg_SR.shape[0], n_bkg_SR, replace=False)
    idx_bkg_SB = np.random.choice(data_bkg_SB.shape[0], n_bkg_SB, replace=False)

    print(f'Preparing dataset from {npy_dirs}')
    for npy_dir in npy_dirs:

        data_sig_SR = np.load(os.path.join(npy_dir, 'sig_in_SR-data.npy'))
        data_sig_SB = np.load(os.path.join(npy_dir, 'sig_in_SB-data.npy'))
        data_bkg_SR = np.load(os.path.join(npy_dir, 'bkg_in_SR-data.npy'))
        data_bkg_SB = np.load(os.path.join(npy_dir, 'bkg_in_SB-data.npy'))

        new_data_SR = np.concatenate([
            data_sig_SR[idx_sig_SR],
            data_bkg_SR[idx_bkg_SR],
        ], axis=0)

        if data_SR is None:
            data_SR = new_data_SR
        else:
            data_SR = np.concatenate([data_SR, new_data_SR], axis=0)

        new_label_SR = np.zeros(n_sig_SR + n_bkg_SR)
        new_label_SR[:n_sig_SR] = 1

        if label_SR is None:
            label_SR = new_label_SR
        else:
            label_SR = np.concatenate([label_SR, new_label_SR])

        new_data_SB = np.concatenate([
            data_sig_SB[idx_sig_SB],
            data_bkg_SB[idx_bkg_SB],
        ], axis=0)

        if data_SB is None:
            data_SB = new_data_SB
        else:
            data_SB = np.concatenate([data_SB, new_data_SB], axis=0)

        new_label_SB = np.zeros(n_sig_SB + n_bkg_SB)
        new_label_SB[:n_sig_SB] = 1

        if label_SB is None:
            label_SB = new_label_SB
        else:
            label_SB = np.concatenate([label_SB, new_label_SB])

    return data_SR, label_SR, data_SB, label_SB

In [5]:
def load_samples(path):
    root, _ = os.path.splitext(path)
    X = np.load(f'{root}-data.npy')
    Y = np.load(f'{root}-label.npy')
    return X, Y

In [6]:
def sculpting_sensitivity(SR_eff, SB_eff, B):
    # SR_eff: background efficiency in signal region
    # SB_eff: background efficiency in sideband region
    # B: number of background events in signal region
    nS = B * (SR_eff - SB_eff)
    nB = B * SB_eff
    return nS / nB**0.5

In [7]:
def get_fpr_thresholds(y_true, y_scores):
    # transform the input to numpy array
    y_true = np.asarray(y_true)
    y_scores = np.asarray(y_scores).reshape(-1)

    # obtain the thresholds
    thresholds = np.sort(np.unique(y_scores))
    
    # get negatives index
    negatives = (y_true == 0)
    negatives_count = np.sum(negatives)
    
    # scores_matrix shape: (n_thresholds, n_samples)
    scores_matrix = y_scores >= thresholds[:, None]
    fp_matrix = np.sum(scores_matrix & negatives, axis=1)
    
    fpr_list = fp_matrix / negatives_count
    
    return fpr_list, thresholds


def get_threshold_from_fpr(fpr, th, passing_rate=0.01):
    # th 由小到大，fpr 由大到小
    passing_rate_idx = (fpr > passing_rate).sum()
    return th[passing_rate_idx]

def get_SRfpr_from_SBfpr(X_SRSB, y_SRSB, model_name, bkg_effs=[0.1]):
    # get the fpr in signal region from the fpr in sideband region
    # fpr: false positive rate, background efficiency

    save_model_name = f'./CNN_models/last_model_CWoLa_hunting_{model_name}/'
    loaded_model = tf.keras.models.load_model(save_model_name)

    X_SR, X_SB = X_SRSB
    y_SR, y_SB = y_SRSB

    y_prob_SB = loaded_model.predict(X_SB)
    fpr_SB, th_SB = get_fpr_thresholds(y_SB == 1, y_prob_SB)

    y_prob_SR = loaded_model.predict(X_SR)
    fpr_SR, th_SR = get_fpr_thresholds(y_SR == 1, y_prob_SR)


    bkg_effs_SR = []
    for bkg_eff in bkg_effs:
        th_SB_bkg_eff = get_threshold_from_fpr(fpr_SB, th_SB, bkg_eff)
        n_th = (th_SR < th_SB_bkg_eff).sum()
        bkg_effs_SR.append(fpr_SR[n_th])

    return bkg_effs_SR

## Original

In [8]:
config_file = 'config_files/origin_config_01.json'
# config_file = 'config_files/jet_aug_3_config_01.json'
# config_file = 'config_files/pt_jet_aug_3_config_01.json'

with open(config_file) as f:
    config = json.load(f)

train_npy_paths = config['train_npy_paths']
val_npy_paths = config['val_npy_paths']
seed = config['seed']
sensitivity = config['sensitivity']

true_label_path = config['true_label_path']
model_name = config['model_name']
sample_type = config['sample_type']

# Training and validation splitting ratio
r_train, r_val = 0.8, 0.2

n_SR_S, n_SR_B, n_SB_S, n_SB_B = compute_nevent_in_SR_SB(sensitivity=sensitivity)

train_nevents = int(n_SR_S * r_train), int(n_SB_S * r_train), int(n_SR_B * r_train), int(n_SB_B * r_train)
X_train_SR, y_train_SR, X_train_SB, y_train_SB  = get_SR_SB_sample_from(train_npy_paths, train_nevents, seed=seed)

Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 0.0, SB: 0.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']


In [9]:
SB_effs = [0.1, 0.01, 0.001]
SR_effs = get_SRfpr_from_SBfpr((X_train_SR, X_train_SB), (y_train_SR, y_train_SB), model_name, SB_effs)

for SB_eff, SR_eff in zip(SB_effs, SR_effs):
    print(f'{SB_eff: .3f} {SR_eff: .4f} {sculpting_sensitivity(SR_eff, SB_eff, n_SR_B): .1f}')

2024-05-23 10:51:00.555258: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-23 10:51:01.183991: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46699 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:3b:00.0, compute capability: 8.6
2024-05-23 10:51:04.269608: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401


 32/506 [>.............................] - ETA: 2s

2024-05-23 10:51:05.906906: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


474/474 [==============================] - 2s 5ms/step
 0.100  0.1238  10.4
 0.010  0.0202  14.1
 0.001  0.0022  5.4


In [10]:
X_test_SR, y_test_SR = load_samples(true_label_path)
X_test_SB, y_test_SB = load_samples('../Sample/HVmodel/data/split_val/25x25/mix_sample_test-SB_25x25.npy')

In [11]:
SR_effs = get_SRfpr_from_SBfpr((X_test_SR, X_test_SB), (y_test_SR, y_test_SB), model_name, SB_effs)

for SB_eff, SR_eff in zip(SB_effs, SR_effs):
    print(f'{SB_eff: .3f} {SR_eff: .4f} {sculpting_sensitivity(SR_eff, SB_eff, n_SR_B): .1f}')

625/625 [==============================] - 3s 4ms/step
 0.100  0.1071  3.1
 0.010  0.0172  9.9
 0.001  0.0006 -1.7


## +3 Jet rotation

In [12]:
# config_file = 'config_files/origin_config_01.json'
config_file = 'config_files/jet_aug_3_config_01.json'
# config_file = 'config_files/pt_jet_aug_3_config_01.json'

with open(config_file) as f:
    config = json.load(f)

train_npy_paths = config['train_npy_paths']
val_npy_paths = config['val_npy_paths']
seed = config['seed']
sensitivity = config['sensitivity']

true_label_path = config['true_label_path']
model_name = config['model_name']
sample_type = config['sample_type']

# Training and validation splitting ratio
r_train, r_val = 0.8, 0.2

n_SR_S, n_SR_B, n_SB_S, n_SB_B = compute_nevent_in_SR_SB(sensitivity=sensitivity)

train_nevents = int(n_SR_S * r_train), int(n_SB_S * r_train), int(n_SR_B * r_train), int(n_SB_B * r_train)
X_train_SR, y_train_SR, X_train_SB, y_train_SB  = get_SR_SB_sample_from(train_npy_paths, train_nevents, seed=seed)

Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 0.0, SB: 0.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25', '../Sample/HVmodel/data/jet_rotation/25x25/01', '../Sample/HVmodel/data/jet_rotation/25x25/02', '../Sample/HVmodel/data/jet_rotation/25x25/03']


In [13]:
SB_effs = [0.1, 0.01, 0.001]
SR_effs = get_SRfpr_from_SBfpr((X_train_SR, X_train_SB), (y_train_SR, y_train_SB), model_name, SB_effs)

for SB_eff, SR_eff in zip(SB_effs, SR_effs):
    print(f'{SB_eff: .3f} {SR_eff: .4f} {sculpting_sensitivity(SR_eff, SB_eff, n_SR_B): .1f}')

1893/1893 [==============================] - 9s 5ms/step
 0.100  0.1421  18.3
 0.010  0.0268  23.1
 0.001  0.0011  0.5


In [14]:
X_test_SR, y_test_SR = load_samples(true_label_path)
X_test_SB, y_test_SB = load_samples('../Sample/HVmodel/data/split_val/25x25/mix_sample_test-SB_25x25.npy')

In [15]:
SR_effs = get_SRfpr_from_SBfpr((X_test_SR, X_test_SB), (y_test_SR, y_test_SB), model_name, SB_effs)

for SB_eff, SR_eff in zip(SB_effs, SR_effs):
    print(f'{SB_eff: .3f} {SR_eff: .4f} {sculpting_sensitivity(SR_eff, SB_eff, n_SR_B): .1f}')

625/625 [==============================] - 3s 5ms/step
 0.100  0.1123  5.4
 0.010  0.0216  16.0
 0.001  0.0003 -3.0


## +3 $p_\text{T}$ smearing + Jet rotation

In [16]:
# config_file = 'config_files/origin_config_01.json'
# config_file = 'config_files/jet_aug_3_config_01.json'
config_file = 'config_files/pt_jet_aug_3_config_01.json'

with open(config_file) as f:
    config = json.load(f)

train_npy_paths = config['train_npy_paths']
val_npy_paths = config['val_npy_paths']
seed = config['seed']
sensitivity = config['sensitivity']

true_label_path = config['true_label_path']
model_name = config['model_name']
sample_type = config['sample_type']

# Training and validation splitting ratio
r_train, r_val = 0.8, 0.2

n_SR_S, n_SR_B, n_SB_S, n_SB_B = compute_nevent_in_SR_SB(sensitivity=sensitivity)

train_nevents = int(n_SR_S * r_train), int(n_SB_S * r_train), int(n_SR_B * r_train), int(n_SB_B * r_train)
X_train_SR, y_train_SR, X_train_SB, y_train_SB  = get_SR_SB_sample_from(train_npy_paths, train_nevents, seed=seed)

Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 0.0, SB: 0.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25', '../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/01', '../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/02', '../Sample/HVmodel/data/pt_smearing_jet_rotation/25x25/03']


In [17]:
SB_effs = [0.1, 0.01, 0.001]
SR_effs = get_SRfpr_from_SBfpr((X_train_SR, X_train_SB), (y_train_SR, y_train_SB), model_name, SB_effs)

for SB_eff, SR_eff in zip(SB_effs, SR_effs):
    print(f'{SB_eff: .3f} {SR_eff: .4f} {sculpting_sensitivity(SR_eff, SB_eff, n_SR_B): .1f}')

1893/1893 [==============================] - 9s 5ms/step
 0.100  0.1318  13.9
 0.010  0.0212  15.4
 0.001  0.0020  4.4


In [18]:
X_test_SR, y_test_SR = load_samples(true_label_path)
X_test_SB, y_test_SB = load_samples('../Sample/HVmodel/data/split_val/25x25/mix_sample_test-SB_25x25.npy')

In [19]:
SR_effs = get_SRfpr_from_SBfpr((X_test_SR, X_test_SB), (y_test_SR, y_test_SB), model_name, SB_effs)

for SB_eff, SR_eff in zip(SB_effs, SR_effs):
    print(f'{SB_eff: .3f} {SR_eff: .4f} {sculpting_sensitivity(SR_eff, SB_eff, n_SR_B): .1f}')

625/625 [==============================] - 3s 5ms/step
 0.100  0.1150  6.5
 0.010  0.0163  8.7
 0.001  0.0011  0.4
